In [3]:
import logging
import sys
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
_=load_dotenv(find_dotenv())
client = OpenAI()
pc = Pinecone()

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['/home/bam/miniconda3/envs/bam/lib/python3.12/site-packages/pinecone_plugins'])
Discovering subpackages in _NamespacePath(['/home/bam/miniconda3/envs/bam/lib/python3.12/site-packages/pinecone_plugins'])
Discovering subpackages in _NamespacePath(['/home/bam/miniconda3/envs/bam/lib/python3.12/site-packages/pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
Installing plugin inference into Pinecone
Installing plugin inference into Pinecone


In [6]:
pc.create_index(
    name="hybrid",
    dimension=1536,
    metric="dotproduct",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [9]:
hybrid_index = pc.Index("hybrid")

In [10]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.node_parser import SentenceSplitter
from IPython.display import Markdown, display

In [22]:
# load documents
documents = SimpleDirectoryReader("./data1", recursive=True).load_data()

In [21]:
def get_embeddings(texts, model="text-embedding-ada-002"):
    response = client.embeddings.create(input=texts, model=model)
    return [data['embedding'] for data in response['data']]
embed_model = get_embeddings

In [24]:
from llama_index.core import StorageContext


vector_store = PineconeVectorStore(
    pinecone_index=hybrid_index,
    add_sparse_vector=True,
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, transformations=[SentenceSplitter(chunk_size=1024, chunk_overlap=20)], storage_context=storage_context
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Upserted vectors: 100%|██████████| 157/157 [00:09<00:00, 16.78it/s]


In [77]:
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine(similarity_top_k=5, model="gpt-4o-mini")
response = query_engine.query("What did Pres. Nelson say about the Cardston Temple?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [78]:
display(Markdown(f"{response}"))

Pres. Nelson mentioned that when he was born, there were six functioning temples in the Church, including one in Cardston, Alberta, Canada.